In [ ]:
# %load game_agent.py
"""Finish all TODO items in this file to complete the isolation project, then
test your agent's strength against a set of known agents using tournament.py
and include the results in your report.
"""
import random


class SearchTimeout(Exception):
    """Subclass base exception for code clarity. """
    pass


def custom_score(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    This should be the best heuristic function for your project submission.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    # This heuristic is an adjustment to the improved_score heuristic from the lectures whereby the agent's reward is 
    # weighted by the total number of available squares in the game. Early on this is high and player should try to 
    # limit their opponent's freedom to avoid being heavily penalised. As the endgame approaches, this agression factor 
    # decreases and the player is rewarded for staying out of trouble (keeping their own options open) 
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    aggression = (len(game.get_legal_moves())) / 25
    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - aggression * opp_moves)
    


def custom_score_2(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
   
    # This heuristic will try to force a board partition early on by awarding a high score to those boxes close to 
    # the main vertical and horizontal
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")

    w, h = game.width / 2., game.height / 2.
    y, x = game.get_player_location(player)
    return float(abs(h - y) + abs(w - x))
    


def custom_score_3(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Note: this function should be called from within a Player instance as
    `self.score()` -- you should not need to call this function directly.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    -------
    float
        The heuristic value of the current game state to the specified player.
    """
    
    # This heuristic is an adjustment to the improved_score heuristic from the lectures whereby the agent's reward is 
    # weighted by an aggression factor that is close to 0 at the start of the game and close to 1 at the end of the game
    # (essentially this is the opposite behaviour to custom_score function above). This means the agent is rewarded for
    # keeping their moves open at the start (avoiding early isolation) and then at the end they are rewarded for 
    # aggressively trying to isolate their opponent.
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")
    
    aggression = (25 - len(game.get_legal_moves())) / 25 
    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - aggression * opp_moves)


class IsolationPlayer:
    """Base class for minimax and alphabeta agents -- this class is never
    constructed or tested directly.

    ********************  DO NOT MODIFY THIS CLASS  ********************

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """
    def __init__(self, search_depth=3, score_fn=custom_score, timeout=10.):
        self.search_depth = search_depth
        self.score = score_fn
        self.time_left = None
        self.TIMER_THRESHOLD = timeout


class MinimaxPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using depth-limited minimax
    search. You must finish and test this player to make sure it properly uses
    minimax to return a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        **************  YOU DO NOT NEED TO MODIFY THIS FUNCTION  *************

        For fixed-depth search, this function simply wraps the call to the
        minimax method, but this method provides a common interface for all
        Isolation agents, and you will replace it in the AlphaBetaPlayer with
        iterative deepening search.

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.minimax(game, self.search_depth)

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move

    

        
        
    def min_value(self, game, depth):
        # Check for timeout in every call to helper functions 
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
        
        # If no available moves then return the terminal utility/score
        if not game.get_legal_moves():
            return self.score(game, self)
                
        # Once minimax has been recursively applied enough times the depth countdown hits zero and we return 
        # the utility/score at that particular depth limitation to then be propagated back up the tree
        if depth == 0:
            return self.score(game, self)
                
        # Initially set v as worst possible result for MIN player then consider every possible move. As these moves
        # backpropagate up a layer, the depth counter is reduced by 1 (terminates when hits zero) and the move we
        # chose is passed to a MAX node. v is keeping track of our current BEST possible play by the MIN player
        # and this should be updated to the minimum of the "old v" and the output of a MAX player's node that our
        # previous MIN player's choice of move m fed into.
        v = float('inf') 
        for m in game.get_legal_moves():
            # Call board evaluation using self.score i.e. custom scoring
            v = self.score(v, self.max_value(game.forecast_move(m), depth-1))
        return v
        
    def max_value(self, game, depth):
        # Check for timeout in every call to helper functions 
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
                
        # If no available moves then return the terminal utility/score
        if not game.get_legal_moves():
            return self.score(game, self)
                
        # Once minimax has been recursively applied enough times the depth countdown hits zero and we return 
        # the utility/score at that particular depth limitation to then be propagated back up the tree
        if depth == 0:
            return self.score(game, self)
                
        # Initially set v as worst possible result for MAX player then consider every possible move. As these moves
        # backpropagate up a layer, the depth counter is reduced by 1 (terminates when hits zero) and the move we
        # chose is passed to a MIN node. v is keeping track of our current BEST possible play by the MAX player
        # and this should be updated to the maximum of the "old v" and the output of a MIN player's node that our
        # previous MAX player's choice of move m fed into.
        for m in game.get_legal_moves():
            # Call board evaluation using self.score i.e. custom scoring
            v = self.score(v, self.min_value(game.forecast_move(m), depth-1))
        return v
        
    def minimax(self, game, depth):
    """Implement depth-limited minimax search algorithm as described in
    the lectures.

    This should be a modified version of MINIMAX-DECISION in the AIMA text.
    https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md

    **********************************************************************
    You MAY add additional methods to this class, or define helper
          functions to implement the required functionality.
    **********************************************************************

    Parameters
    ----------
    game : isolation.Board
        An instance of the Isolation game `Board` class representing the
        current game state

    depth : int
        Depth is an integer representing the maximum number of plies to
        search in the game tree before aborting

    Returns
    -------
    (int, int)
        The board coordinates of the best move found in the current search;
        (-1, -1) if there are no legal moves

    Notes
    -----
        (1) You MUST use the `self.score()` method for board evaluation
            to pass the project tests; you cannot call any other evaluation
            function directly.

        (2) If you use any helper functions (e.g., as shown in the AIMA
            pseudocode) then you must copy the timer check into the top of
            each helper function or else your agent will timeout during
            testing.
    """
    if self.time_left() < self.TIMER_THRESHOLD:
        raise SearchTimeout()
    
    legal_moves = game.get_legal_moves()
    best_score = float('-inf')
    if not legal_moves:
            return (-1, -1)
    
    best_move = legal_moves[0]
    for m in legal_moves:
        # Because we increment the depth below, we are instructing our agent to pick the move that will give our opponent
        # (the MIN player) the highest score (worst for them and best for us). Assuming optimal play by both players this 
        # will then filter down the tree by the recurrent nature of our min and max helper functions (defined above) and 
        # ultimately give us the best possible score i.e. route/moves through the tree
        score = self.min_value(game.forecast_move(m), depth-1)
        if score > best_score:
            best_score = score
            best_move = m
    return m

class AlphaBetaPlayer(IsolationPlayer):
    """Game-playing agent that chooses a move using iterative deepening minimax
    search with alpha-beta pruning. You must finish and test this player to
    make sure it returns a good move before the search time limit expires.
    """

    def get_move(self, game, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        Modify the get_move() method from the MinimaxPlayer class to implement
        iterative deepening search instead of fixed-depth search.

        **********************************************************************
        NOTE: If time_left() < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """
        self.time_left = time_left

        # TODO: finish this function!
        # Initialize the best move so that this function returns something
        # in case the search fails due to timeout
        best_move = (-1, -1)

        try:
            # The try/except block will automatically catch the exception
            # raised when the timer is about to expire.
            return self.alphabeta(game, self.search_depth)

        except SearchTimeout:
            pass  # Handle any actions required after timeout as needed

        # Return the best move from the last completed search iteration
        return best_move

    
        
    def ab_max_value(self, game, depth, alpha, beta):
        # Check for timeout in every call to helper functions 
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
          
        # If no available moves then return the terminal utility/score
        if not game.get_legal_moves():
            return self.score(game, self)
                
        # Once minimax has been recursively applied enough times the depth countdown hits zero and we return 
        # the utility/score at that particular depth limitation to then be propagated back up the tree
        if depth == 0:
            return self.score(game, self)
                
        # Initially set v as worst possible result for MAX player then consider every possible move. As these moves
        # backpropagate up a layer, the depth counter is reduced by 1 (terminates when hits zero) and the move we
        # chose is passed to a MIN node. v is keeping track of our current BEST possible play by the MAX player
        # and this should be updated to the maximum of the "old v" and the output of a MIN player's node that our
        # previous MAX player's choice of move m fed into.
        v = float('-inf') 
        for m in game.get_legal_moves():
            # Call board evaluation using self.score i.e. custom scoring
            v = self.score(v, self.ab_min_value(game.forecast_move(m), depth-1, alpha, beta))
            # The alpha-beta bit is to check every possible move to see if it is greater than beta (the current
            # worst performance for a MIN player). If it is then we should pick this as our choice for the MAX 
            # player. We then want to update alpha to be the maximum of this v and the old value of alpha as this
            # now represents a guaranteed score that MAX can achieve and thus is our current estimate of the MAX
            # player's worst possible performance (assuming no better moves lie ahead further down the tree).
            if v >= beta:
                return v
                alpha = max(alpha, v)
            return v
    
    def ab_min_value(self, game, depth, alpha, beta):
        # Check for timeout in every call to helper functions 
        if self.time_left() < self.TIMER_THRESHOLD:
            raise SearchTimeout()
          
        # If no available moves then return the terminal utility/score
        if not game.get_legal_moves():
            return self.score(game, self)
                
        # Once minimax has been recursively applied enough times the depth countdown hits zero and we return 
        # the utility/score at that particular depth limitation to then be propagated back up the tree
        if depth == 0:
            return self.score(game, self)
                
        # Initially set v as worst possible result for MIN player then consider every possible move. As these moves
        # backpropagate up a layer, the depth counter is reduced by 1 (terminates when hits zero) and the move we
        # chose is passed to a MAX node. v is keeping track of our current BEST possible play by the MIN player
        # and this should be updated to the maximum of the "old v" and the output of a MAX player's node that our
        # previous MIN player's choice of move m fed into.
        v = float('inf') 
        for m in game.get_legal_moves():
            # Call board evaluation using self.score i.e. custom scoring
            v = self.score(v, self.ab_max_value(game.forecast_move(m), depth-1, alpha, beta))
            # The alpha-beta bit is to check every possible move to see if it is less than alpha (the current
            # worst performance for a MAX player). If it is then we should pick this as our choice for the MIN 
            # player. We then want to update beta to be the maximum of this v and the old value of beta as this
            # now represents a guaranteed score that MIN can achieve and thus is our current estimate of the MIN
            # player's worst possible performance (assuming no better moves lie ahead further down the tree).
            if v <= alpha:
                return v
                beta = min(beta, v)
            return v
    
    def alphabeta(self, game, depth, alpha, beta):
    
    if self.time_left() < self.TIMER_THRESHOLD:
        raise SearchTimeout()
    
    legal_moves = game.get_legal_moves()
    best_score = float('-inf')
    if not legal_moves:
            return (-1, -1)
    
    best_move = legal_moves[0]
    for m in legal_moves:
        # Because we increment the depth below, we are instructing our agent to pick the move that will give our opponent
        # (the MIN player) the highest score (worst for them and best for us). Assuming optimal play by both players this 
        # will then filter down the tree by the recurrent nature of our min and max helper functions (defined above) and 
        # ultimately give us the best possible score i.e. route/moves through the tree
        score = self.min_value(game.forecast_move(m), depth-1)
        if score > best_score:
            best_score = score
            best_move = m
        alpha = max(alpha, m)
    return m
